# Zillow Housing Times Series Analysis



# 1. Business Understanding
## 1.1 Overview

With their recent successful real estate investment in California, our client Homegates Property Group seeks to expand their listings in the state of New York, which happens to be the state with the most Fortune 500 company headquarters in the United States and the world. Our client believes the trends and contributing factors that they saw in California, especially the continued job growth will also positively impact the home values in New York. The team is looking for recommendations on top 5 zipcodes to invest in New York, and this analysis will also provide them with short-term vs. long-term investment decisions.

## 1.2 Problem Statement

The goal of this analysis is to identify the top 5 zipcodes for our client to invest in New York. The team is not quite familiar with the East Coast real estate market, and therefore, has asked to take risk factor into consideration. The results from this analysis will provide them with the forecast of next 10 year mean house values in the top 5 zipcodes as well as expected ROI in 1 year, 3 years, 5 years, and 10 years.

## 1.3 Objective

 1. To determine the expected ROI yield in 1 year, 3 years, 5 years, and 10 years.
 2. To identify the top 5 zipcodes for our client to invest in New York based on the ROI yields.
 
## 1.4 Metric of Success



# 2. Data Understanding
## 2.1 Data Overview
The dataset for this analysis comes from Zillow Research, which contains the median home sales prices in 14,723 individual zipcodes from April 1996 through April 2018. 
Each row represents a unique zip code indexed with RegioinID, and contains location info and median housing sales prices for each month.

## 2.2 Data Description

RegionID: Unique index, 58196 through 753844

RegionName: Unique Zip Code, 1001 through 99901

City: City in which the zip code is located

State: State in which the zip code is located

Metro: Metropolitan Area in which the zip code is located

CountyName: County in which the zip code is located

SizeRank: Numerical rank of size of zip code, ranked 1 through 14723

1996-04 through 2018-04: refers to the median housing sales values for April 1996 through April 2018, that is 265 data points of monthly data for each zip code



## 2.3 Load the Data/Filtering for Chosen Zipcodes

In [1]:
#import the necessary libraries
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


In [2]:
#load the data into a dataframe and preview
df = pd.read_csv('data/electric_production.csv')
df.head()

,DATE,IPG2211A2N
0,1939-01-01,3.3335
1,1939-02-01,3.3590
2,1939-03-01,3.4353
3,1939-04-01,3.4607
4,1939-05-01,3.4607


In [3]:
df.tail()

,DATE,IPG2211A2N
1001,2022-06-01,104.5230
1002,2022-07-01,115.5747
1003,2022-08-01,114.2425
1004,2022-09-01,100.5098
1005,2022-10-01,92.3498


In [4]:
#general description of the data
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DATE        1006 non-null   object 
 1   IPG2211A2N  1006 non-null   float64
dtypes: float64(1), object(1)
memory usage: 15.8+ KB


The dataset has 14723 rows and 272 columns with 268 numerical datatypes and 4 categorical datatypes.

In [ ]:
#check for duplicates 

# Step 2: Data Preprocessing

In [ ]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results